In [6]:
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('Data/stand_scaled_data.csv')
features = data.columns.tolist()
features.remove('y')

In [8]:
data_sample = data.sample(5000).reset_index()

In [11]:
def CVSVM(df, n_splits=5,  rando_state=2021, features=features, if_print=True, 
                  *args, **kwargs):
    # Prepare KStratifiedKFOLD
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=rando_state)
    
    # Make copy of data
    data = df.copy()
    
    # Prepare empty lists
    train_results = []
    test_results = []
    preds = []
    indicies = []
    # Prepare int to count fold s
    fold_number = 1
       
    for train, test in kf.split(data.index.values, data['y']):
        # Prepare KNN model 
        model = SVC(C=1, # wartość kary za błąd klasyfikacji (hiperparametr modelu
                  cache_size=500, 
                  kernel='linear', # Wybór kernela (narazie liniowy)
                  max_iter=-1, 
                  probability=True, # czy funkcja ma zwracać prawdopodobieństwa?
                  tol=0.001, 
                  verbose=False, *args, **kwargs)
        model.fit(data.loc[train, features], data.loc[train, 'y'])
        
        # Make predictions
        train_preds = model.predict(data.loc[train, features])
        test_preds = model.predict(data.loc[test, features])
        preds.append(test_preds)
        
        indicies.append(df.iloc[test].index.tolist().copy())
        
        # Prepare ROC_AUC score
        train_roc = metrics.roc_auc_score(data.loc[train, 'y'], train_preds)
        test_roc = metrics.roc_auc_score(data.loc[test, 'y'], test_preds)
        
        # Add ROC_AUC to lis
        train_results.append(train_roc)
        test_results.append(test_roc)
        
        if if_print:
            print(f'FOLD NUMBER: {fold_number}')
            print(f'ROC_AUC ON TRAIN SCORE {train_roc}')
            print(f'ROC_AUC ON TEST SCORE {test_roc}')
                  
        fold_number += 1 
        
    return train_results, test_results, preds, indicies

In [13]:
model = CVSVM(data_sample)
# Model bardzo wolny nawet na samplu danych, słabe wyniki

FOLD NUMBER: 1
ROC_AUC ON TRAIN SCORE 0.7716273090006636
ROC_AUC ON TEST SCORE 0.7681861793759132
FOLD NUMBER: 2
ROC_AUC ON TRAIN SCORE 0.7708024987070311
ROC_AUC ON TEST SCORE 0.7603196755271593
FOLD NUMBER: 3
ROC_AUC ON TRAIN SCORE 0.7710268743452164
ROC_AUC ON TEST SCORE 0.7492422425260037
FOLD NUMBER: 4
ROC_AUC ON TRAIN SCORE 0.7683452620138855
ROC_AUC ON TEST SCORE 0.7561874441222006
FOLD NUMBER: 5
ROC_AUC ON TRAIN SCORE 0.7616607427113543
ROC_AUC ON TEST SCORE 0.763931699202505
